In [1]:
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image as img
import os
from scipy.io import savemat,loadmat

2023-12-20 16:52:41.932787: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 16:52:43.591413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:

def gettensor(folder_path,file_name):
    im=img.open(os.path.join(folder_path,file_name)).convert('L')
    im=im.resize((1100,1100),img.LANCZOS)
    tens=tf.constant(im,dtype=tf.float32)
    tens=tf.reshape(tens,(1,1100,1100,1))/tf.reduce_max(tens)
    del im
    return tens

def prepare(list,folder_path,file_name_list):
    x=gettensor(folder_path,file_name_list[list[0]])
    for i in list[1:]:
        sup=gettensor(folder_path,file_name_list[i])
        x=tf.concat((x,sup),axis=0)
    return x

In [6]:
train_folder="/home/featurize/data/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train"
test_folder="/home/featurize/data/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test"
train_name=pd.read_csv("/home/featurize/work/data/train_shuffled.csv")["X_ray_image_name"]
train_name=np.array(train_name)
test_name=pd.read_csv("/home/featurize/work/data/test_shuffled.csv")["X_ray_image_name"]
test_name=np.array(test_name)

In [7]:
l_train_y=pd.read_csv("/home/featurize/work/data/train_shuffled.csv")["Label"]
l_test_y=pd.read_csv("/home/featurize/work/data/test_shuffled.csv")["Label"]

def numberize(l_y):
    y=[]
    for lab in l_y:
        if lab=="Normal":
            y.append(0)
        else:
            y.append(1)
    return y

train_y=tf.constant(numberize(l_train_y),dtype=tf.int32)
test_y=tf.constant(numberize(l_test_y),dtype=tf.int32)

In [8]:
model_path="/home/featurize/work/System/M1_sec"
class ConvM(Model):
    def __init__(self):
        super(ConvM,self).__init__()
        #注意：每一层不只是做运算，而是有它自己的权重和偏置，所以每前向传播一步，都必须设一个新层
        self.conv2d1=tf.keras.layers.Conv2D(16,kernel_size=(5,5),activation="relu")
        self.conv2d2=tf.keras.layers.Conv2D(16,kernel_size=(5,5),activation="relu")
        self.conv2d3=tf.keras.layers.Conv2D(32,kernel_size=(5,5),activation="relu")
        self.conv2d4=tf.keras.layers.Conv2D(32,kernel_size=(5,5),activation="relu")
        self.maxpool1=tf.keras.layers.MaxPool2D(pool_size=(4,4))
        self.maxpool2=tf.keras.layers.MaxPool2D(pool_size=(4,4))
        self.flatten=tf.keras.layers.Flatten()
        self.dropout1=tf.keras.layers.Dropout(0.2)
        self.dropout2=tf.keras.layers.Dropout(0.2)
        self.dense1=tf.keras.layers.Dense(256,activation="relu")
        self.dense2=tf.keras.layers.Dense(2,activation="softmax")
        
    def call(self,x):
        x=self.conv2d1(x)
        x=self.conv2d2(x)
        x=self.maxpool1(x)
        x=self.dropout1(x)
        
        x=self.conv2d3(x)
        x=self.conv2d4(x)
        x=self.maxpool2(x)
        x=self.dropout2(x)
        
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.dense2(x)
        return x

try:
    model=tf.keras.models.load_model(model_path)
except IOError:
    model=ConvM()
    print("Model Initialized")

In [9]:
loss_obj=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer=tf.keras.optimizers.RMSprop(0.001,0.9)

In [10]:
train_loss=tf.keras.metrics.Mean(name="train_loss")
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss=tf.keras.metrics.Mean(name="test_loss")
test_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [11]:
@tf.function
def train_step(image,label):
    with tf.GradientTape() as tape:
        predict=model(image,training=True)
        loss=loss_obj(label,predict)
        
    grad=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grad,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label,predict)

In [12]:
@tf.function
def test_step(image,label):
    predict=model(image,training=False)
    loss=loss_obj(label,predict)
    
    test_loss(loss)
    test_accuracy(label,predict)

In [13]:
if __name__=="__main__":
    os.system('featurize event create --title "通讯测试" --content "这是内容"')
    verbose_dir="/home/featurize/work/verbose.txt"
    file=open(verbose_dir,"w")
    file.write("New Trainning\n------------------\n")
    file.close()
    
    flag=True
    
    counter=0
    test_range=100
    
    EPOCHS=500
    for epoch in range(EPOCHS):
        train_loss.reset_states()
        test_loss.reset_states()
        train_accuracy.reset_states()
        test_accuracy.reset_states()
        
        train_list=np.random.randint(0,len(train_y),200)
        train_tensor=prepare(train_list,train_folder,train_name)
        
        test_list=np.random.randint(0,len(test_y),test_range)
        
        if flag:
            j=0
            for i in train_list:
                image=tf.reshape(train_tensor[j],(1,1100,1100,1))
                j=j+1
                label=train_y[i]
                train_step(image,label)
                if j%500==0 and j!=0 :
                    file=open(verbose_dir,"a")
                    file.write(f"Training(epoch={epoch+1}), Accuracy=%{train_accuracy.result()*100}\n")
                    file.close()
                    model.save(model_path,save_format="tf")
                
        for i in test_list:
            image=gettensor(test_folder,test_name[i])
            label=test_y[i]
            test_step(image,label)
        if flag==True and test_accuracy.result()>=0.87:
            flag=False
            test_range=300
        elif flag==False and test_accuracy.result()>=0.84 and counter<=4:
            counter=counter+1
            file=open(verbose_dir,'a')
            file.write(f"Validating. Test accuracy = %{test_accuracy.result()*100}.\n")
            file.close()
        elif flag==False and test_accuracy.result()>=0.84 and counter>4:
            file=open(verbose_dir,'a')
            file.write(f"Validating finished. final Test accuracy = %{test_accuracy.result()*100}. Training finished.\n")
            file.close()
            os.system('featurize event create --title "已实现训练目标" --content "这是内容"')
            break
        elif flag==False and test_accuracy.result()<0.84:
            flag=True
            counter=0
            test_range=100
        else:
            pass
            
        model.save(model_path,save_format="tf")
        print("Model Saved.")
        
        file=open(verbose_dir,"a")
        file.write(f"Epoch {epoch+1}, "
             f"Train Loss: {train_loss.result()}, "
             f"Train Accuracy: %{train_accuracy.result()*100}, "
             f"Test Loss: {test_loss.result()}, "
             f"Test Accuracy: %{test_accuracy.result()*100}.\n")
        file.close()
    
    file=open(verbose_dir,'a')
    file.write("All Epochs Are Done.\n")
    file.close()
    
os.system('featurize event create --title "实例已归还" --content "这是内容"')
os.system("featurize instance release")

/tmp/ipykernel_18877/2133538872.py:3: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im=im.resize((1100,1100),img.LANCZOS)
2023-12-20 16:53:24.088429: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inconv_m/StatefulPartitionedCall/StatefulPartitionedCall/dropout/StatefulPartitionedCall/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-12-20 16:53:25.133149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-20 16:53:27.724625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-20 16:53:29.674630: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd3c10a5c00 initialized for platform CUDA (this does not guarantee that X

INFO:tensorflow:Assets written to: /home/featurize/work/System/M1_sec/assets


INFO:tensorflow:Assets written to: /home/featurize/work/System/M1_sec/assets


Model Saved.


/tmp/ipykernel_18877/2133538872.py:3: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im=im.resize((1100,1100),img.LANCZOS)


...

INFO:tensorflow:Assets written to: /home/featurize/work/System/M1_sec/assets


INFO:tensorflow:Assets written to: /home/featurize/work/System/M1_sec/assets


Model Saved.


/tmp/ipykernel_18877/2133538872.py:3: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im=im.resize((1100,1100),img.LANCZOS)


KeyboardInterrupt: 